# Exhaustive Search for Small N Values

Focus on N=2-6 which contribute most to score and have smallest search space.
For each N, try many angle combinations and optimize positions.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import itertools
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 25

# Tree geometry constants
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_polygon(cx, cy, angle_deg):
    """Get tree polygon at position (cx, cy) with given angle."""
    angle_rad = np.radians(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    
    # Rotate and translate
    x = TX * cos_a - TY * sin_a + cx
    y = TX * sin_a + TY * cos_a + cy
    
    return Polygon(zip(x, y))

def get_bounding_box_side(trees):
    """Get the side length of the bounding box for a list of tree polygons."""
    all_poly = unary_union(trees)
    bounds = all_poly.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def check_overlap(trees):
    """Check if any trees overlap."""
    for i in range(len(trees)):
        for j in range(i+1, len(trees)):
            if trees[i].intersects(trees[j]):
                intersection = trees[i].intersection(trees[j])
                if intersection.area > 1e-10:
                    return True
    return False

print("Functions defined.")

Functions defined.


In [2]:
# Load current baseline to compare
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].str.strip('s').astype(float)
df['y'] = df['y'].str.strip('s').astype(float)
df['deg'] = df['deg'].str.strip('s').astype(float)
df['n'] = df['id'].str[:3].astype(int)

# Get current scores for small N
baseline_scores = {}
for n in range(1, 13):
    group = df[df['n'] == n]
    trees = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    score = side**2 / n
    baseline_scores[n] = {'side': side, 'score': score}
    print(f"N={n}: side={side:.6f}, score={score:.6f}")

print(f"\nTotal score for N=1-12: {sum(s['score'] for s in baseline_scores.values()):.6f}")

N=1: side=0.813173, score=0.661250
N=2: side=0.949504, score=0.450779
N=3: side=1.142031, score=0.434745
N=4: side=1.290806, score=0.416545
N=5: side=1.443692, score=0.416850
N=6: side=1.548438, score=0.399610
N=7: side=1.673104, score=0.399897
N=8: side=1.755921, score=0.385407
N=9: side=1.867280, score=0.387415
N=10: side=1.940696, score=0.376630
N=11: side=2.033002, score=0.375736
N=12: side=2.114873, score=0.372724

Total score for N=1-12: 5.077588


In [3]:
def optimize_positions_for_angles(n, angles, max_iter=100):
    """
    Given fixed angles for n trees, optimize their positions to minimize bounding box.
    Uses scipy.optimize.minimize with BFGS.
    """
    def objective(params):
        # params = [x1, y1, x2, y2, ..., xn, yn]
        trees = []
        for i in range(n):
            cx, cy = params[2*i], params[2*i+1]
            trees.append(get_tree_polygon(cx, cy, angles[i]))
        
        # Check for overlaps - return large penalty if overlapping
        if check_overlap(trees):
            return 1000.0
        
        return get_bounding_box_side(trees)
    
    # Initialize positions in a grid pattern
    init_positions = []
    grid_size = int(np.ceil(np.sqrt(n)))
    for i in range(n):
        row, col = i // grid_size, i % grid_size
        init_positions.extend([col * 1.5, row * 1.5])
    
    # Optimize
    result = minimize(objective, init_positions, method='Nelder-Mead', 
                     options={'maxiter': max_iter, 'xatol': 1e-6, 'fatol': 1e-6})
    
    return result.fun, result.x

print("Optimizer defined.")

Optimizer defined.


In [4]:
def search_best_angles(n, angle_step=15, max_iter=50):
    """
    Search for best angle combinations for n trees.
    For small n, try many angle combinations.
    """
    best_side = float('inf')
    best_config = None
    
    # For n=2, we can try more angles
    if n == 2:
        angles_to_try = range(0, 360, angle_step)
    else:
        # For larger n, use fewer angles to keep search tractable
        angles_to_try = range(0, 360, angle_step * 2)
    
    # Generate angle combinations
    # For efficiency, we fix first tree at angle 0 (due to rotational symmetry)
    count = 0
    total = len(list(itertools.product(angles_to_try, repeat=n-1)))
    
    for other_angles in itertools.product(angles_to_try, repeat=n-1):
        angles = [0] + list(other_angles)
        
        side, positions = optimize_positions_for_angles(n, angles, max_iter=max_iter)
        
        if side < best_side:
            best_side = side
            best_config = {'angles': angles, 'positions': positions}
            print(f"  N={n}: New best side={side:.6f} (angles={angles[:3]}...)")
        
        count += 1
        if count % 1000 == 0:
            print(f"  Progress: {count}/{total} ({100*count/total:.1f}%)")
    
    return best_side, best_config

print("Search function defined.")

Search function defined.


In [5]:
# The baseline is already well-optimized. Let's analyze the baseline configurations\n# and try to improve them with local search instead of searching from scratch.\n\nprint("Analyzing baseline configurations for N=2-6...\\n")\n\nfor n in range(2, 7):\n    group = df[df['n'] == n]\n    print(f"N={n}:")\n    for _, row in group.iterrows():\n        print(f"  Tree {row['id']}: x={row['x']:.4f}, y={row['y']:.4f}, deg={row['deg']:.2f}")\n    \n    trees = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]\n    side = get_bounding_box_side(trees)\n    print(f"  Side: {side:.6f}, Score: {side**2/n:.6f}\\n")


=== Searching N=2 ===
Baseline: side=0.949504, score=0.450779
  N=2: New best side=1.227857 (angles=[0, 0]...)
  N=2: New best side=1.222019 (angles=[0, 40]...)


  N=2: New best side=1.196992 (angles=[0, 170]...)
  N=2: New best side=1.166587 (angles=[0, 175]...)
  N=2: New best side=1.121166 (angles=[0, 185]...)
  N=2: New best side=1.108547 (angles=[0, 195]...)
  N=2: New best side=1.105404 (angles=[0, 200]...)



Best found: side=1.105404, score=0.610959
Improvement: -0.160180


In [ ]:
# Search for better configurations for N=3
print("\n=== Searching N=3 ===")
print(f"Baseline: side={baseline_scores[3]['side']:.6f}, score={baseline_scores[3]['score']:.6f}")

best_side_3, best_config_3 = search_best_angles(3, angle_step=30, max_iter=50)
best_score_3 = best_side_3**2 / 3

print(f"\nBest found: side={best_side_3:.6f}, score={best_score_3:.6f}")
print(f"Improvement: {baseline_scores[3]['score'] - best_score_3:.6f}")

In [ ]:
# Search for better configurations for N=4
print("\n=== Searching N=4 ===")
print(f"Baseline: side={baseline_scores[4]['side']:.6f}, score={baseline_scores[4]['score']:.6f}")

best_side_4, best_config_4 = search_best_angles(4, angle_step=45, max_iter=50)
best_score_4 = best_side_4**2 / 4

print(f"\nBest found: side={best_side_4:.6f}, score={best_score_4:.6f}")
print(f"Improvement: {baseline_scores[4]['score'] - best_score_4:.6f}")

In [ ]:
# Summary of improvements found
print("\n=== Summary ===")
improvements = {}

for n, best_side, best_config in [(2, best_side_2, best_config_2), 
                                   (3, best_side_3, best_config_3),
                                   (4, best_side_4, best_config_4)]:
    if best_config is not None:
        best_score = best_side**2 / n
        baseline = baseline_scores[n]['score']
        improvement = baseline - best_score
        improvements[n] = {'side': best_side, 'score': best_score, 'improvement': improvement, 'config': best_config}
        print(f"N={n}: baseline={baseline:.6f}, best={best_score:.6f}, improvement={improvement:.6f}")

total_improvement = sum(imp['improvement'] for imp in improvements.values() if imp['improvement'] > 0)
print(f"\nTotal potential improvement: {total_improvement:.6f}")

In [ ]:
# If we found improvements, update the submission
if total_improvement > 0:
    print("\nUpdating submission with improved configurations...")
    
    # Load current submission
    df = pd.read_csv('/home/submission/submission.csv')
    
    for n, imp in improvements.items():
        if imp['improvement'] > 0:
            config = imp['config']
            angles = config['angles']
            positions = config['positions']
            
            # Update rows for this N
            for i in range(n):
                idx = df[df['id'] == f'{n:03d}_{i}'].index[0]
                df.loc[idx, 'x'] = f"s{positions[2*i]:.18f}"
                df.loc[idx, 'y'] = f"s{positions[2*i+1]:.18f}"
                df.loc[idx, 'deg'] = f"s{angles[i]:.18f}"
            
            print(f"Updated N={n}")
    
    # Save
    df.to_csv('/home/submission/submission.csv', index=False)
    print("Saved updated submission.")
else:
    print("No improvements found.")

In [ ]:
# Verify the new score
print("\nVerifying new score...")
df = pd.read_csv('/home/submission/submission.csv')
df['x'] = df['x'].str.strip('s').astype(float)
df['y'] = df['y'].str.strip('s').astype(float)
df['deg'] = df['deg'].str.strip('s').astype(float)
df['n'] = df['id'].str[:3].astype(int)

total_score = 0
for n in range(1, 201):
    group = df[df['n'] == n]
    trees = [get_tree_polygon(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
    side = get_bounding_box_side(trees)
    score = side**2 / n
    total_score += score

print(f"New total score: {total_score:.6f}")
print(f"Previous score: 70.659437")
print(f"Improvement: {70.659437 - total_score:.6f}")